# 특징

- swin_base_patch4_window12_384
- Scheduler : CosineAnnealingLR
- normal train/test split
- early stopping

 # What I can do more

 - NO K-Fold
 - [Done] Backbone Freezing
 - EarlyStopping
 - More model Ensemble
 - Feature Engineering

# Install Required Libraries

In [1]:
# ! pip install git+https://github.com/rwightman/pytorch-image-models
! pip install -q -U wandb albumentations timm

In [2]:
import os
import gc
import cv2
import copy
import time
import random
from PIL import Image
import matplotlib.pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# Pytorch Image Model Library
import timm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import warnings

warnings.filterwarnings("ignore")
import os

# For descriptive error messages
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"



# CONFIG

In [3]:
ROOT_DIR = "../input/petfinder-pawpularity-score"
MODEL_DIR = "../input/pretrained-model"
TRAIN_DIR = f"{ROOT_DIR}/train"
TEST_DIR = f"{ROOT_DIR}/test"



In [4]:
CONFIG = dict(
    seed=42,
    backbone="swin_base_patch4_window12_384",
    train_batch_size=16,
    valid_batch_size=32,
    img_size=384,
    fine_tune_epochs=1,
    epochs=100,
    learning_rate=1e-4,
    scheduler="CosineAnnealingLR",
    min_lr=1e-6,
    T_max=100,
    weight_decay=1e-6,
    n_accumulate=1,
    use_KFold=False,
    n_fold=5,
    num_classes=1,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    competition="PetFinder",
    _wandb_kernel_="deb",
    early_stopping=True,
    early_stopping_step=10,
)


## wandb config

In [5]:
import wandb

os.environ["WANDB_API_KEY"] = "d60a4af56f6cd9cccec7d9da1dbced7960b61310"
# if want to run in offline mode
# os.environ["WANDB_MODE"] = "dryrun"
wandb.login(key="d60a4af56f6cd9cccec7d9da1dbced7960b61310")
wandb.init(project="petfinder-pawpularity-score", entity="jiwon7258")
wandb.run.name = "swin384_1fold_finetuning-enabled"



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jiwon7258 (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jiwon/.netrc
2021-12-13 12:57:38.479679: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jiwon/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2021-12-13 12:57:38.480969: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Set Seed for Reproducibility

In [6]:
def set_seed(seed=42):
    # numpy
    np.random.seed(seed)
    # python
    random.seed(seed)
    # torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)


set_seed(CONFIG["seed"])



# Read the Data

In [7]:
def get_train_file_path(id):
    return f"{TRAIN_DIR}/{id}.jpg"


def get_test_file_path(id):
    return f"{TEST_DIR}/{id}.jpg"



In [8]:
df = pd.read_csv(f"{ROOT_DIR}/train.csv")
# file_path에 해당하는 column을 만든다
df["file_path"] = df["Id"].apply(get_train_file_path)



In [9]:
df.head()


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,file_path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...


In [10]:
# feature_cols를 통해 사용할 feature 목록을 관리한다
feature_cols = [
    col for col in df.columns if col not in ["Id", "Pawpularity", "file_path"]
]


# Create Folds

Pawpularity는 0~100 사이의 정수 값을 가진다. Stratifed

In [11]:
def create_folds(df, n_s=5, n_grp=None):
    df["kfold"] = -1

    if n_grp is None:
        skf = KFold(n_splits=n_s, random_state=CONFIG["seed"])
        target = df["Pawpularity"]
    else:
        skf = StratifiedKFold(n_splits=n_s, shuffle=True, random_state=CONFIG["seed"])
        # Pawpularity를 구간별로, n_grp 수만큼 자른다
        # 따라서 Pawpularity와 grp의 히스토그램 분포는 동일하다
        df["grp"] = pd.cut(df["Pawpularity"], n_grp, labels=False)
        target = df["grp"]

    # n_grp의 분포를 기반으로 StratifiedKFold를 진행한다
    for fold_no, (t, v) in enumerate(skf.split(target, target)):
        df.loc[v, "kfold"] = fold_no

    df = df.drop("grp", axis=1)
    return df


In [12]:
df = create_folds(df, n_s=CONFIG["n_fold"], n_grp=14)
df.head()


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,file_path,kfold
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...,0
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...,2
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...,0
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...,3
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...,4


Pawpularity는 0~100까지의 정수로 이루어져 있다. 이 분포 그대로 KFold를 진행하지 않는다. 대신 n_grp개 만큼, 일정한 길이별로 구간을 나눈 후, 이 분포를 이용하여 KFold를 진행한다.

```df.Pawpularity.hist(bins=14) == df.grp.hist(bins=14)```

![Pawpularity Histogram](./img/paupularity.png)

![이미지](./img/pawpularity_and_grp_hist.png)



# Dataset Class 

In [13]:
class PawpularityDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir = root_dir
        self.df = df
        self.file_names = df["file_path"].values  # numpy array
        self.targets = df["Pawpularity"].values  # numpy array
        self.transforms = transforms

    # 데이터 프레임의 길이를 반환
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)  # numpy array
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.targets[index]

        if self.transforms:
            img = self.transforms(image=img)["image"]

        # 이미지 데이터, target label
        return img, target


# Augmentations

In [14]:
data_transforms = {
    "train": A.Compose(
        [
            # 리사이징
            A.Resize(CONFIG["img_size"], CONFIG["img_size"]),
            # 가로 반전
            A.HorizontalFlip(p=0.5),
            # 랜덤 밝기/대조
            A.RandomBrightnessContrast(p=0.3),
            # 정규화
            A.Normalize(),
            ToTensorV2(),
        ],
        p=1.0,
    ),
    "valid": A.Compose(
        [A.Resize(CONFIG["img_size"], CONFIG["img_size"]), A.Normalize(), ToTensorV2()]
    ),
}


# Create Model

In [15]:
class PawpularityModel(nn.Module):
    def __init__(self, backbone, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        o = self.backbone(torch.randn(1, 3, CONFIG["img_size"], CONFIG["img_size"]))
        self.n_features = o.shape[-1]
        # freeze backbone layer
        for (name, param) in self.backbone.named_parameters():
            if "layers.3.blocks" not in name:
                param.requires_grad = False
        self.backbone.norm.requires_grad_()
        self.fc = nn.Linear(self.n_features, CONFIG["num_classes"])

    def forward(self, images):
        features = self.backbone(images)  # features = (batch size, embedding_size)
        output = self.fc(features)  # outputs = (batch_size, num_classes)
        return output


model = PawpularityModel(CONFIG["backbone"])
# model.load_state_dict(torch.load('RMSE4.8957_epoch27.bin'))
model.to(CONFIG["device"])


PawpularityModel(
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (0): BasicLayer(
        dim=128, input_resolution=(96, 96), depth=2
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=128, out_features=384, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=128, out_features=128, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
   

# Loss Function

In [16]:
def criterion(outputs: torch.tensor, targets: torch.tensor):
    return torch.sqrt(nn.MSELoss()(outputs.view(-1), targets.view(-1)))


# Training Function

In [17]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    # train 모드로 변경
    model.train()

    # for the Mixed Precision
    # Pytorch 예제 : https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples
    scaler = amp.GradScaler()

    dataset_size = 0
    running_loss = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (images, targets) in bar:
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)

        batch_size = images.size(0)

        with amp.autocast(enabled=True):
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss = loss / CONFIG["n_accumulate"]

        # loss를 Scale
        # Scaled Grdients를 계산(call)하기 위해 scaled loss를 backward()
        scaler.scale(loss).backward()

        if (step + 1) % CONFIG["n_accumulate"] == 0:
            # scaler.step() first unscales the gradients of the optimizer's assigned params.
            # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
            # otherwise, optimizer.step() is skipped.
            scaler.step(optimizer)
            # Updates the scale for next iteration.
            scaler.update()

            # zero the parameter gradients
            optimizer.zero_grad()

            # change learning rate by Scheduler
            if scheduler is not None:
                scheduler.step()

        # loss.item()은 loss를 Python Float으로 반환
        # loss.item()은 batch data의 average loss이므로, sum of loss를 구하기 위해 batch_size를 곱해준다
        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        bar.set_postfix(
            Epoch=epoch, Train_Loss=epoch_loss, LR=optimizer.param_groups[0]["lr"]
        )

    # Garbage Collector
    gc.collect()

    return epoch_loss


# Validation Function

In [18]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0

    TARGETS = []
    PREDS = []

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (images, targets) in bar:
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)

        batch_size = images.size(0)

        outputs = model(images)
        loss = criterion(outputs, targets)

        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        # 실시간으로 정보를 표시하기 위한 epoch loss
        epoch_loss = running_loss / dataset_size

        PREDS.append(outputs.view(-1).cpu().detach().numpy())
        TARGETS.append(targets.view(-1).cpu().detach().numpy())

        bar.set_postfix(
            Epoch=epoch, Valid_Loss=epoch_loss, LR=optimizer.param_groups[0]["lr"]
        )

    TARGETS = np.concatenate(TARGETS)
    PREDS = np.concatenate(PREDS)
    # 실제 epoch loss는 다음과 같이 계산한다
    val_rmse = mean_squared_error(TARGETS, PREDS, squared=False)

    gc.collect()

    return epoch_loss, val_rmse


# Prepare DataLoader and Scheduler

In [ ]:
"""
fold : 여러개의 fold 중, validation set으로 이용할 fold #
fold 이외의 folds는 train set으로 이용된다
"""
def prepare_loaders(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = PawpularityDataset(
        TRAIN_DIR, df_train, transforms=data_transforms["train"]
    )
    valid_dataset = PawpularityDataset(
        TRAIN_DIR, df_valid, transforms=data_transforms["valid"]
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG["train_batch_size"],
        num_workers=0,
        shuffle=True,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=CONFIG["valid_batch_size"],
        num_workers=0,
        shuffle=False,
        pin_memory=True,
    )

    return train_loader, valid_loader


In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG["scheduler"] == "CosineAnnealingLR":
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=CONFIG["T_max"], eta_min=CONFIG["min_lr"]
        )
    elif CONFIG["scheduler"] == "CosineAnnealingWarmRestarts":
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=CONFIG["T_0"], eta_min=CONFIG["min_lr"]
        )
    elif CONFIG["scheduler"] == None:
        return None

    return scheduler


## Create Dataloader

In [ ]:
train_loader = dict()
valid_loader = dict()

for f in range(CONFIG["n_fold"]):
    train_, valid_ = prepare_loaders(f)
    train_loader[f] = train_
    valid_loader[f] = valid_

## Define Optimizer and Scheduler

In [ ]:
optimizer = optim.Adam(
    model.parameters(), lr=CONFIG["learning_rate"], weight_decay=CONFIG["weight_decay"]
)
scheduler = fetch_scheduler(optimizer)

# Control Training Func

In [ ]:
def run_training(
    model,
    optimizer,
    scheduler,
    device,
    num_epochs,
    use_KFold: bool,
    use_finetune: bool,
    early_stopping=True,
    early_stopping_step=10,
):
    # To automatically log graidents
    wandb.watch(model, log_freq=100)

    if torch.cuda.is_available():
        print("[INFO] Using GPU:{}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_rmse = np.inf
    history = defaultdict(list)
    early_stop_counter = 0

    # K-fold Validation : epoch마다 fold가 달라진다
    # num_epochs만큼, train과 val을 실행한다
    for epoch in range(1, num_epochs + 1):
        gc.collect()

        # use if K-Fold validation : fold == valdiation fold #
        if use_KFold:
            fold = epoch % CONFIG["n_fold"]
        else:
            fold = 0

        # train & valid one epoch
        print(f"Validation Fold : {fold}")

        train_epoch_loss = train_one_epoch(
            model,
            optimizer,
            scheduler,
            dataloader=train_loader[fold],
            device=device,
            epoch=epoch,
        )
        
        val_epoch_loss, val_epoch_rmse = valid_one_epoch(
            model, valid_loader[fold], device=device, epoch=epoch
        )

        finetune = ""
        if use_finetune:
            finetune = "finetune_"

        history[f"{finetune}Train Loss"].append(train_epoch_loss)
        history[f"{finetune}Valid Loss"].append(val_epoch_loss)
        history[f"{finetune}Valid RMSE"].append(val_epoch_rmse)

        # Log the metrics
        wandb.log({f"{finetune}Train Loss": train_epoch_loss})
        wandb.log({f"{finetune}Valid Loss": val_epoch_loss})
        wandb.log({f"{finetune}Valid RMSE": val_epoch_rmse})

        print(f"Valid RMSE : {val_epoch_rmse}")

        # deep copy the model
        if val_epoch_rmse <= best_epoch_rmse:
            early_stop_counter = 0
            print(
                f"Validation Loss improved( {best_epoch_rmse} ---> {val_epoch_rmse}  )"
            )
            best_epoch_rmse = val_epoch_rmse
            # run.summary['Best RMSE'] = best_epoch_rmse
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "{}RMSE{:.4f}_epoch{:.0f}.bin".format(
                finetune, best_epoch_rmse, epoch
            )
            torch.save(model.state_dict(), PATH)
            torch.save(model.state_dict(), f"{finetune}best_wts.bin")
            # Save a model file from the current directory
            wandb.save(PATH)
            print(f"Model Saved")
        
        elif early_stopping:
            early_stop_counter += 1
            if early_stop_counter > early_stopping_step:
                break

        print()

    end = time.time()
    time_elapsed = end - start
    print(
        "Training complete in {:.0f}h {:.0f}m {:.0f}s".format(
            time_elapsed // 3600,
            (time_elapsed % 3600) // 60,
            (time_elapsed % 3600) % 60,
        )
    )
    print("Best RMSE: {:.4f}".format(best_epoch_rmse))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history


# Fine Tuning


In [ ]:
model, history = run_training(
    model,
    optimizer,
    scheduler,
    device=CONFIG["device"],
    num_epochs=CONFIG["fine_tune_epochs"],
    use_KFold=CONFIG["use_KFold"],
    use_finetune=False,
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],

)


# Whole Tuning (Training)

## Load FineTuned Weight

In [ ]:
MODEL_PATH = "best_wts.bin"
model.load_state_dict(torch.load(MODEL_PATH, map_location=CONFIG["device"]))

## Unfreeze All layer

In [ ]:
for param in model.parameters():
    param.requires_grad = True


## Start Training

In [ ]:
model, history = run_training(
    model,
    optimizer,
    scheduler,
    device=CONFIG["device"],
    num_epochs=CONFIG["epochs"],
    use_Kfold=CONFIG["use_KFold"],
    use_finetune=False,
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
)


# TEST and SUBMIT

## Load Model

In [ ]:
TEST_MODEL_PATH = f"{MODEL_DIR}/RMSE4.8957_epoch27.bin"
model.load_state_dict(torch.load(TEST_MODEL_PATH, map_location=CONFIG["device"]))


## Test Dataset

In [ ]:
class PawpularityTestDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir = root_dir
        self.df = df
        self.file_names = df["file_path"].values  # numpy array
        self.transforms = transforms

    # 데이터 프레임의 길이를 반환
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)  # numpy array
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        Id = self.df.Id[index]

        if self.transforms:
            img = self.transforms(image=img)["image"]

        # 이미지 데이터, target label
        return img, Id


## Prepare Test Data

In [ ]:
df_test = pd.read_csv(f"{ROOT_DIR}/test.csv")
# file_path에 해당하는 column을 만든다
df_test["file_path"] = df_test["Id"].apply(get_test_file_path)

In [ ]:
test_dataset = PawpularityTestDataset(
    root_dir=TRAIN_DIR, df=df_test, transforms=data_transforms["valid"]
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=CONFIG["valid_batch_size"],
    shuffle=False,
    pin_memory=True,
    num_workers=4,
)

In [ ]:
df_test.head()

In [ ]:
print(len(test_loader))
with torch.no_grad():
    model.eval()

    total_size = 0
    outputs = []
    Ids = []

    bar = tqdm(enumerate(test_loader), total=len(test_loader))

    for step, (images, Id) in bar:
        images = images.cuda()

        batch_size = images.shape[0]

        output = model(images)

        total_size += batch_size

        outputs.append(output.cpu())
        Ids.extend(Id)

outputs = np.concatenate(outputs, axis=0).flatten()

In [ ]:
submission = pd.DataFrame({"Id": Ids, "Pawpularity": outputs})
submission.to_csv("submission.csv", index=False)

In [ ]:
submission